#### Finding matching park with taxi zone

In [1]:
import geopandas as gpd
import pandas as pd
import folium 
import matplotlib.pyplot as plt
import mapclassify

manhattan_parks_gdf = gpd.read_file("manhattan_parks_geocoded_full.geojson")
#manhattan_taxi_zones_gdf = gpd.read_file("manhattan_taxi_zones.geojson")


In [2]:
manhattan_parks_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [3]:
pd.set_option('display.max_columns', None)
#display all columns not truncated

In [4]:
manhattan_parks_gdf.head()

,id,name,website,park_area,geometry
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770..."
1,relation/2389631,Governors Island,https://www.nps.gov/gois/index.htm,696227.552870,"POLYGON ((-74.01540 40.69306, -74.01556 40.693..."
2,relation/3690816,Highbridge Park,https://www.nycgovparks.org/parks/highbridge-p...,521310.282769,"MULTIPOLYGON (((-73.93110 40.84702, -73.93112 ..."
3,relation/4099553,Sara D. Roosevelt Park,https://www.nycgovparks.org/parks/sara-d-roose...,36496.652767,"MULTIPOLYGON (((-73.99377 40.71801, -73.99426 ..."
4,relation/5625630,Fort Tryon Park,https://www.nycgovparks.org/parks/fort-tryon-park,281836.601376,"MULTIPOLYGON (((-73.92808 40.86526, -73.92816 ..."


In [5]:
manhattan_parks_gdf['boundary'] = manhattan_parks_gdf.boundary

In [6]:
manhattan_parks_gdf= manhattan_parks_gdf.set_geometry('boundary')

In [8]:
busyness_taxizones= gpd.read_file("yellow_taxi_busyness_details_hourly.geojson")

KeyboardInterrupt: 

In [6]:
#manhattan_parks_gdf= manhattan_parks_gdf.to_crs(epsg=4326)
busyness_taxizones=busyness_taxizones.to_crs(epsg=32618)

In [7]:
busyness_taxizones.crs

<Projected CRS: EPSG:32618>
Name: WGS 84 / UTM zone 18N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 78°W and 72°W, northern hemisphere between equator and 84°N, onshore and offshore. Bahamas. Canada - Nunavut; Ontario; Quebec. Colombia. Cuba. Ecuador. Greenland. Haiti. Jamaica. Panama. Turks and Caicos Islands. United States (USA). Venezuela.
- bounds: (-78.0, 0.0, -72.0, 84.0)
Coordinate Operation:
- name: UTM zone 18N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
# Calculate area taxi zones
busyness_taxizones['taxi_zone_area'] = busyness_taxizones.geometry.area

In [9]:
busyness_taxizones.head(5)

,location_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,zone,geometry,taxi_zone_area
0,100,2022,1,1,10,5,True,Winter,True,6,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667
1,100,2022,1,1,11,5,True,Winter,True,9,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667
2,100,2022,1,1,12,5,True,Winter,True,4,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667
3,100,2022,1,1,16,5,True,Winter,True,24,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667
4,100,2022,1,1,17,5,True,Winter,True,4,Garment District,"MULTIPOLYGON (((585491.231 4511548.637, 585451...",351170.494667


Change busyness_taxizone CRS back to 4326 to spatial join

In [10]:
busyness_taxizones=busyness_taxizones.to_crs(epsg=4326)

In [11]:
manhattan_parks_gdf=manhattan_parks_gdf.to_crs(epsg=4326)

In [12]:
busyness_taxizones=busyness_taxizones.rename(columns={'location_id':'taxi_zone_id','passenger_count_number':'taxi_zone_busyness'})

In [13]:
busyness_taxizones=busyness_taxizones.rename(columns={'zone':'taxi_zone'})

In [14]:
manhattan_parks_gdf=manhattan_parks_gdf.rename(columns={'id':'park_id','name':'park_name'})

In [15]:
busyness_taxizones.head()

,taxi_zone_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,geometry,taxi_zone_area
0,100,2022,1,1,10,5,True,Winter,True,6,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667
1,100,2022,1,1,11,5,True,Winter,True,9,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667
2,100,2022,1,1,12,5,True,Winter,True,4,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667
3,100,2022,1,1,16,5,True,Winter,True,24,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667
4,100,2022,1,1,17,5,True,Winter,True,4,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667


In [16]:
manhattan_parks_gdf.head()

,park_id,park_name,website,park_area,geometry
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770..."
1,relation/2389631,Governors Island,https://www.nps.gov/gois/index.htm,696227.552870,"POLYGON ((-74.01540 40.69306, -74.01556 40.693..."
2,relation/3690816,Highbridge Park,https://www.nycgovparks.org/parks/highbridge-p...,521310.282769,"MULTIPOLYGON (((-73.93110 40.84702, -73.93112 ..."
3,relation/4099553,Sara D. Roosevelt Park,https://www.nycgovparks.org/parks/sara-d-roose...,36496.652767,"MULTIPOLYGON (((-73.99377 40.71801, -73.99426 ..."
4,relation/5625630,Fort Tryon Park,https://www.nycgovparks.org/parks/fort-tryon-park,281836.601376,"MULTIPOLYGON (((-73.92808 40.86526, -73.92816 ..."


#### Spatial join the parks with taxi zone

In [17]:
# Perform a spatial join to find parks within each taxi zone
parks_with_zones = gpd.sjoin(manhattan_parks_gdf, busyness_taxizones, how='inner', predicate='intersects')

print(parks_with_zones[['park_name', 'taxi_zone_id']])


                                  park_name taxi_zone_id
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
0                             John Jay Park          140
..                                      ...          ...
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142
471  103 W 62nd St, New York, NY 10023, USA          142

[895969 rows x 2 columns]


In [106]:
e= parks_with_zones.nunique()
e

park_id                        458
park_name                      435
website                        153
park_area                      413
geometry                       458
index_right                 113735
taxi_zone_id                    65
year                             1
month                           12
day                             31
hour                            24
day_of_week                      7
weekend                          2
season                           4
isHoliday?                       2
taxi_zone_busyness             166
taxi_zone                       65
taxi_zone_area                  65
remaining_taxi_zone_area       477
park_proportion                477
park_busyness                21497
dtype: int64

In [102]:
busyness_taxizones.head()

,taxi_zone_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,geometry,taxi_zone_area
0,100,2022,1,1,10,5,True,Winter,True,6,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667
1,100,2022,1,1,11,5,True,Winter,True,9,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667
2,100,2022,1,1,12,5,True,Winter,True,4,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667
3,100,2022,1,1,16,5,True,Winter,True,24,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667
4,100,2022,1,1,17,5,True,Winter,True,4,Garment District,"MULTIPOLYGON (((-73.98729 40.75045, -73.98777 ...",351170.494667


In [20]:
parks_with_zones.head()

,park_id,park_name,website,park_area,geometry,index_right,taxi_zone_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,taxi_zone_area
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17495,140,2022,4,6,12,2,False,Spring,False,18,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17496,140,2022,4,6,13,2,False,Spring,False,5,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17497,140,2022,4,6,17,2,False,Spring,False,22,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17498,140,2022,4,6,18,2,False,Spring,False,65,Lenox Hill East,692556.6829
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17499,140,2022,4,6,19,2,False,Spring,False,10,Lenox Hill East,692556.6829


#### Calculating remaining taxi zone area

In [21]:
# Calculate the remaining area of the taxi zone
parks_with_zones['remaining_taxi_zone_area'] = parks_with_zones['taxi_zone_area'] - parks_with_zones['park_area']

# Calculate the proportion of the taxi zone area that is covered by the park
parks_with_zones['park_proportion'] = parks_with_zones['park_area'] / parks_with_zones['taxi_zone_area']

# Calculate the busyness score for the park
parks_with_zones['park_busyness'] = parks_with_zones['park_proportion'] * parks_with_zones['taxi_zone_busyness']

# Display the resulting DataFrame
print(parks_with_zones[['park_name', 'park_id','park_area','taxi_zone_area','park_proportion', 'taxi_zone_busyness','park_busyness']])


                                  park_name           park_id    park_area  \
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
0                             John Jay Park  relation/1751075  9506.859431   
..                                      ...               ...          ...   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   
471  103 W 62nd St, New York, NY 10023, USA   node/7447580924   313.654849   

     taxi_zone_area  park_proportion  taxi_zone_busyness  park_

#### Find intersecting parks and taxi zones

In [22]:
parks_with_zones.head()

,park_id,park_name,website,park_area,geometry,index_right,taxi_zone_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17495,140,2022,4,6,12,2,False,Spring,False,18,Lenox Hill East,692556.6829,683049.823468,0.013727,0.247089
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17496,140,2022,4,6,13,2,False,Spring,False,5,Lenox Hill East,692556.6829,683049.823468,0.013727,0.068636
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17497,140,2022,4,6,17,2,False,Spring,False,22,Lenox Hill East,692556.6829,683049.823468,0.013727,0.301998
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17498,140,2022,4,6,18,2,False,Spring,False,65,Lenox Hill East,692556.6829,683049.823468,0.013727,0.892268
0,relation/1751075,John Jay Park,N/A,9506.859431,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",17499,140,2022,4,6,19,2,False,Spring,False,10,Lenox Hill East,692556.6829,683049.823468,0.013727,0.137272


In [25]:
parks_with_zones_individual=parks_with_zones[parks_with_zones['park_name']== 'East River Waterfront Esplanade']
parks_with_zones_individual.head(30)

,park_id,park_name,website,park_area,geometry,index_right,taxi_zone_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112471,88,2022,10,7,14,4,False,Autumn,False,4,Financial District South,243402.199452,200687.631991,0.17549,0.701959
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112470,88,2022,10,7,13,4,False,Autumn,False,6,Financial District South,243402.199452,200687.631991,0.17549,1.052938
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112469,88,2022,10,7,11,4,False,Autumn,False,5,Financial District South,243402.199452,200687.631991,0.17549,0.877448
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112468,88,2022,10,7,10,4,False,Autumn,False,1,Financial District South,243402.199452,200687.631991,0.17549,0.175490
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112467,88,2022,10,6,7,3,False,Autumn,False,6,Financial District South,243402.199452,200687.631991,0.17549,1.052938
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112466,88,2022,10,6,6,3,False,Autumn,False,2,Financial District South,243402.199452,200687.631991,0.17549,0.350979
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112465,88,2022,10,6,19,3,False,Autumn,False,1,Financial District South,243402.199452,200687.631991,0.17549,0.175490
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112464,88,2022,10,6,18,3,False,Autumn,False,2,Financial District South,243402.199452,200687.631991,0.17549,0.350979
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112463,88,2022,10,6,17,3,False,Autumn,False,11,Financial District South,243402.199452,200687.631991,0.17549,1.930386
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112462,88,2022,10,5,8,2,False,Autumn,False,3,Financial District South,243402.199452,200687.631991,0.17549,0.526469


In [42]:
parks_with_zones_individual['taxi_zone_id'].value_counts()

taxi_zone_id
87     1930
232    1677
88     1584
45     1393
209    1387
Name: count, dtype: int64

In [43]:
parks_with_zones_filtered=parks_with_zones[['park_id','park_name','geometry','year','month','day','hour']]
parks_with_zones_filtered

,park_id,park_name,geometry,year,month,day,hour
0,relation/1751075,John Jay Park,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",2022,4,6,12
0,relation/1751075,John Jay Park,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",2022,4,6,13
0,relation/1751075,John Jay Park,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",2022,4,6,17
0,relation/1751075,John Jay Park,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",2022,4,6,18
0,relation/1751075,John Jay Park,"POLYGON ((-73.94901 40.76981, -73.94944 40.770...",2022,4,6,19
...,...,...,...,...,...,...,...
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",POINT (-73.98352 40.77130),2022,6,13,21
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",POINT (-73.98352 40.77130),2022,6,12,19
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",POINT (-73.98352 40.77130),2022,6,12,18
471,node/7447580924,"103 W 62nd St, New York, NY 10023, USA",POINT (-73.98352 40.77130),2022,6,12,17


there are 60,062 rows duplicate based on same park id, year, month, day, hour

In [44]:
#checking for duplicated rows
duplicated_rows = parks_with_zones_filtered[parks_with_zones_filtered.duplicated()]

#display duplicate rows
print("Duplicate Rows:")
print(duplicated_rows)


Duplicate Rows:
              park_id                                  park_name  \
2    relation/3690816                            Highbridge Park   
2    relation/3690816                            Highbridge Park   
2    relation/3690816                            Highbridge Park   
2    relation/3690816                            Highbridge Park   
2    relation/3690816                            Highbridge Park   
..                ...                                        ...   
417    way/1276741632  385 Riverside Dr, New York, NY 10025, USA   
417    way/1276741632  385 Riverside Dr, New York, NY 10025, USA   
417    way/1276741632  385 Riverside Dr, New York, NY 10025, USA   
417    way/1276741632  385 Riverside Dr, New York, NY 10025, USA   
417    way/1276741632  385 Riverside Dr, New York, NY 10025, USA   

                                              geometry  year month day hour  
2    MULTIPOLYGON (((-73.93110 40.84702, -73.93112 ...  2022     8  19   20  
2    MULTIP

In [45]:
parks_taxi_info_datetime_hourly=parks_with_zones

In [46]:
parks_with_zones.columns

Index(['park_id', 'park_name', 'website', 'park_area', 'geometry',
       'index_right', 'taxi_zone_id', 'year', 'month', 'day', 'hour',
       'day_of_week', 'weekend', 'season', 'isHoliday?', 'taxi_zone_busyness',
       'taxi_zone', 'taxi_zone_area', 'remaining_taxi_zone_area',
       'park_proportion', 'park_busyness'],
      dtype='object')

Example of resolve the duplicate issue

In [55]:
EastRiver_data= parks_taxi_info_datetime_hourly[(parks_taxi_info_datetime_hourly['park_id'] == 'relation/16066428') & (parks_taxi_info_datetime_hourly['park_name'] == 'East River Waterfront Esplanade') & ((parks_taxi_info_datetime_hourly['month'] == '10')) & ((parks_taxi_info_datetime_hourly['day'] == '7'))& ((parks_taxi_info_datetime_hourly['hour'] == '14'))]
EastRiver_data

,park_id,park_name,website,park_area,geometry,index_right,taxi_zone_id,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",112471,88,2022,10,7,14,4,False,Autumn,False,4,Financial District South,2.434022e+05,2.006876e+05,0.175490,0.701959
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",110596,87,2022,10,7,14,4,False,Autumn,False,1,Financial District North,3.883466e+05,3.456320e+05,0.109991,0.109991
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",49304,209,2022,10,7,14,4,False,Autumn,False,1,Seaport,2.598693e+05,2.171547e+05,0.164369,0.164369
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",96728,45,2022,10,7,14,4,False,Autumn,False,1,Chinatown,6.917331e+05,6.490185e+05,0.061750,0.061750
27,relation/16066428,East River Waterfront Esplanade,N/A,42714.567461,"MULTIPOLYGON (((-74.00333 40.70564, -74.00355 ...",60633,232,2022,10,7,14,4,False,Autumn,False,1,Two Bridges/Seward Park,1.413727e+06,1.371013e+06,0.030214,0.030214


In [56]:
EastRiver_filtered = EastRiver_data[['park_id', 'park_name','park_area','geometry','year','month', 'day', 'hour','day_of_week', 'weekend', 'season', 'isHoliday?','taxi_zone_busyness','taxi_zone_id','taxi_zone','taxi_zone_area','remaining_taxi_zone_area','park_proportion','park_busyness']]
EastRiver_dissolved=EastRiver_filtered.dissolve(by=['park_id','park_name','park_area','year','month','day','hour','day_of_week','weekend','season','isHoliday?'], aggfunc='sum')
EastRiver_dissolved

,,,,,,,,,,,geometry,taxi_zone_busyness,taxi_zone_id,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness
park_id,park_name,park_area,year,month,day,hour,day_of_week,weekend,season,isHoliday?,,,,,,,,
relation/16066428,East River Waterfront Esplanade,42714.567461,2022,10,7,14,4,False,Autumn,False,"MULTIPOLYGON (((-74.00384 40.70530, -74.00325 ...",8,888720945232,Financial District SouthFinancial District Nor...,2.997079e+06,2.783506e+06,0.541814,1.068283


In [57]:
EastRiver_dissolved=EastRiver_dissolved.reset_index()

In [58]:
EastRiver_dissolved['taxi_zone_ids']=EastRiver_data.groupby(['park_id', 'park_name','year','month', 'day', 'hour'])['taxi_zone_id'].apply(lambda x:', '.join(x)).reset_index()['taxi_zone_id']
EastRiver_dissolved['taxi_zones']=EastRiver_data.groupby(['park_id', 'park_name','year','month', 'day', 'hour'])['taxi_zone'].apply(lambda x:', '.join(x)).reset_index()['taxi_zone']

EastRiver_dissolved


,park_id,park_name,park_area,year,month,day,hour,day_of_week,weekend,season,isHoliday?,geometry,taxi_zone_busyness,taxi_zone_id,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness,taxi_zone_ids,taxi_zones
0,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,10,7,14,4,False,Autumn,False,"MULTIPOLYGON (((-74.00384 40.70530, -74.00325 ...",8,888720945232,Financial District SouthFinancial District Nor...,2.997079e+06,2.783506e+06,0.541814,1.068283,"88, 87, 209, 45, 232","Financial District South, Financial District N..."


In [59]:
EastRiver_dissolved.drop(['taxi_zone_id','taxi_zone'],axis=1)

,park_id,park_name,park_area,year,month,day,hour,day_of_week,weekend,season,isHoliday?,geometry,taxi_zone_busyness,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness,taxi_zone_ids,taxi_zones
0,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,10,7,14,4,False,Autumn,False,"MULTIPOLYGON (((-74.00384 40.70530, -74.00325 ...",8,2.997079e+06,2.783506e+06,0.541814,1.068283,"88, 87, 209, 45, 232","Financial District South, Financial District N..."


#### Now working with all the data

In [60]:
parks_with_zones_filtered = parks_with_zones[['park_id', 'park_name','park_area','geometry','year','month', 'day', 'hour','day_of_week', 'weekend', 'season', 'isHoliday?','taxi_zone_busyness','taxi_zone_id','taxi_zone','taxi_zone_area','remaining_taxi_zone_area','park_proportion','park_busyness']]

In [76]:
# Group by 'park_id' and 'park_name' and apply union_geometry
#filtered_group= parks_with_zones_filtered.groupby(['park_id', 'park_name'])

In [77]:
#from shapely.ops import unary_union

In [78]:
#def union_geometry(x):
    #return unary_union(x)



In [79]:
# Apply union_geometry to each group and combine results
#dissolved_and_unioned = filtered_group['geometry'].apply(lambda geom: union_geometry(geom))

In [80]:
#dissolved_and_unioned_df = dissolved_and_unioned.reset_index()

In [81]:
# Merge the unified geometries back with the dissolved DataFrame
#parks_with_zones_filtered = parks_with_zones_filtered.drop(columns='geometry').merge(dissolved_and_unioned_df, on=['park_id', 'park_name'])

In [61]:
#type(parks_with_zones_filtered)

geopandas.geodataframe.GeoDataFrame

In [62]:
parks_with_zones_filtered = gpd.GeoDataFrame(parks_with_zones_filtered, geometry='geometry')

In [70]:
parks_with_zones_dissolved=parks_with_zones_filtered.dissolve(by=['park_id','park_name','park_area','year','month','day','hour','day_of_week','weekend','season','isHoliday?'], aggfunc='sum')
parks_with_zones_dissolved

geometry  \
park_id          park_name                 park_area   year month day hour day_of_week weekend season isHoliday?                                                      
node/11457630561 Kowsky Plaza              313.654849  2022 1     1   10   5           True    Winter True                               POINT (-74.01693 40.71186)   
                                                                      11   5           True    Winter True                               POINT (-74.01693 40.71186)   
                                                                      12   5           True    Winter True                               POINT (-74.01693 40.71186)   
                                                                      16   5           True    Winter True                               POINT (-74.01693 40.71186)   
                                                                      17   5           True    Winter True                               POINT (-74.01693 40.71186)   
...                                                                                                                                                             ...   
way/997819338    Andrew Haswell Green Park 3009.158663 2022 9     9   18   4           False   Autumn False       POLYGON ((-73.95728 40.75996, -73.95733 40.759...   
                                                                      19   4           False   Autumn False       POLYGON ((-73.95728 40.75996, -73.95733 40.759...   
                                                                      20   4           False   Autumn False       POLYGON ((-73.95728 40.75996, -73.95733 40.759...   
                                                                      8    4           False   Autumn False       POLYGON ((-73.95728 40.75996, -73.95733 40.759...   
                                                                      9    4           False   Autumn False       POLYGON ((-73.95728 40.75996, -73.95733 40.759...   

                                                                                                                  taxi_zone_busyness  \
park_id          park_name                 park_area   year month day hour day_of_week weekend season isHoliday?                       
node/11457630561 Kowsky Plaza              313.654849  2022 1     1   10   5           True    Winter True                        16   
                                                                      11   5           True    Winter True                         8   
                                                                      12   5           True    Winter True                         5   
                                                                      16   5           True    Winter True                        19   
                                                                      17   5           True    Winter True                         2   
...                                                                                                                              ...   
way/997819338    Andrew Haswell Green Park 3009.158663 2022 9     9   18   4           False   Autumn False                        2   
                                                                      19   4           False   Autumn False                       20   
                                                                      20   4           False   Autumn False                        9   
                                                                      8    4           False   Autumn False                       19   
                                                                      9    4           False   Autumn False                       18   

                                                                                                                 taxi_zone_id  \
park_id          park_name                 park_area   year month day hour day_of_wee

In [71]:
parks_with_zones_dissolved= parks_with_zones_dissolved.reset_index()

In [73]:
parks_with_zones_dissolved['taxi_zone_ids']=parks_with_zones_filtered.groupby(['park_id', 'park_name','year','month', 'day', 'hour'])['taxi_zone_id'].apply(lambda x:', '.join(x)).reset_index()['taxi_zone_id']
parks_with_zones_dissolved['taxi_zones']=parks_with_zones_filtered.groupby(['park_id', 'park_name','year','month', 'day', 'hour'])['taxi_zone'].apply(lambda x:', '.join(x)).reset_index()['taxi_zone']


In [74]:
parks_with_zones_before_dissolve=parks_with_zones_filtered.nunique()
parks_with_zones_before_dissolve

park_id                       458
park_name                     435
park_area                     413
geometry                      458
year                            1
month                          12
day                            31
hour                           24
day_of_week                     7
weekend                         2
season                          4
isHoliday?                      2
taxi_zone_busyness            166
taxi_zone_id                   65
taxi_zone                      65
taxi_zone_area                 65
remaining_taxi_zone_area      477
park_proportion               477
park_busyness               21497
dtype: int64

In [75]:
parks_with_zones_after_dissolve= parks_with_zones_dissolved.nunique()
parks_with_zones_after_dissolve

park_id                       458
park_name                     435
park_area                     413
year                            1
month                          12
day                            31
hour                           24
day_of_week                     7
weekend                         2
season                          4
isHoliday?                      2
geometry                      484
taxi_zone_busyness            205
taxi_zone_id                  220
taxi_zone                     220
taxi_zone_area                147
remaining_taxi_zone_area      571
park_proportion               570
park_busyness               39253
taxi_zone_ids                 220
taxi_zones                    220
dtype: int64

Due to the nature of dissolve(), duplicate geometries might occur due to slight precision issues, need to use unary_union()

In [76]:
# Group by 'park_id' and 'park_name' and apply union_geometry
grouped = parks_with_zones_dissolved.groupby(['park_id', 'park_name'])

In [77]:
from shapely.ops import unary_union

In [78]:
def union_geometry(x):
    return unary_union(x)



In [79]:
# Apply union_geometry to each group and combine results
dissolved_and_unioned = grouped['geometry'].apply(lambda geom: union_geometry(geom))

In [80]:
dissolved_and_unioned_df = dissolved_and_unioned.reset_index()

In [87]:
# Merge the unified geometries back with the dissolved DataFrame
parks_with_zones_dissolved = parks_with_zones_dissolved.drop(columns='geometry').merge(dissolved_and_unioned_df, on=['park_id', 'park_name'])

In [88]:
print(f"Unique park IDs: {parks_with_zones_dissolved['park_id'].nunique()}")
print(f"Unique geometries: {parks_with_zones_dissolved['geometry'].nunique()}")

Unique park IDs: 458
Unique geometries: 458


In [89]:
parks_with_zones_dissolved.nunique()


park_id                       458
park_name                     435
park_area                     413
year                            1
month                          12
day                            31
hour                           24
day_of_week                     7
weekend                         2
season                          4
isHoliday?                      2
taxi_zone_busyness            205
taxi_zone_id                  220
taxi_zone                     220
taxi_zone_area                147
remaining_taxi_zone_area      571
park_proportion               570
park_busyness               39253
taxi_zone_ids                 220
taxi_zones                    220
geometry                      458
dtype: int64

In [90]:
parks_with_zones_dissolved

,park_id,park_name,park_area,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone_id,taxi_zone,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness,taxi_zone_ids,taxi_zones,geometry
0,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,10,5,True,Winter,True,16,13,Battery Park City,457069.67484,456756.019991,0.000686,0.010980,13,Battery Park City,POINT (-74.01693 40.71186)
1,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,11,5,True,Winter,True,8,13,Battery Park City,457069.67484,456756.019991,0.000686,0.005490,13,Battery Park City,POINT (-74.01693 40.71186)
2,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,12,5,True,Winter,True,5,13,Battery Park City,457069.67484,456756.019991,0.000686,0.003431,13,Battery Park City,POINT (-74.01693 40.71186)
3,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,16,5,True,Winter,True,19,13,Battery Park City,457069.67484,456756.019991,0.000686,0.013038,13,Battery Park City,POINT (-74.01693 40.71186)
4,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,17,5,True,Winter,True,2,13,Battery Park City,457069.67484,456756.019991,0.000686,0.001372,13,Battery Park City,POINT (-74.01693 40.71186)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835902,way/997819338,Andrew Haswell Green Park,3009.158663,2022,9,9,18,4,False,Autumn,False,2,140,Lenox Hill East,692556.68290,689547.524236,0.004345,0.008690,140,Lenox Hill East,"POLYGON ((-73.95786 40.75948, -73.95761 40.759..."
835903,way/997819338,Andrew Haswell Green Park,3009.158663,2022,9,9,19,4,False,Autumn,False,20,140,Lenox Hill East,692556.68290,689547.524236,0.004345,0.086900,140,Lenox Hill East,"POLYGON ((-73.95786 40.75948, -73.95761 40.759..."
835904,way/997819338,Andrew Haswell Green Park,3009.158663,2022,9,9,20,4,False,Autumn,False,9,140,Lenox Hill East,692556.68290,689547.524236,0.004345,0.039105,140,Lenox Hill East,"POLYGON ((-73.95786 40.75948, -73.95761 40.759..."
835905,way/997819338,Andrew Haswell Green Park,3009.158663,2022,9,9,8,4,False,Autumn,False,19,140,Lenox Hill East,692556.68290,689547.524236,0.004345,0.082555,140,Lenox Hill East,"POLYGON ((-73.95786 40.75948, -73.95761 40.759..."


In [92]:
parks_with_zones_dissolved=parks_with_zones_dissolved.drop(['taxi_zone','taxi_zone_id'], axis=1)
parks_with_zones_dissolved

,park_id,park_name,park_area,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness,taxi_zone_ids,taxi_zones,geometry
0,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,10,5,True,Winter,True,16,457069.67484,456756.019991,0.000686,0.010980,13,Battery Park City,POINT (-74.01693 40.71186)
1,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,11,5,True,Winter,True,8,457069.67484,456756.019991,0.000686,0.005490,13,Battery Park City,POINT (-74.01693 40.71186)
2,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,12,5,True,Winter,True,5,457069.67484,456756.019991,0.000686,0.003431,13,Battery Park City,POINT (-74.01693 40.71186)
3,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,16,5,True,Winter,True,19,457069.67484,456756.019991,0.000686,0.013038,13,Battery Park City,POINT (-74.01693 40.71186)
4,node/11457630561,Kowsky Plaza,313.654849,2022,1,1,17,5,True,Winter,True,2,457069.67484,456756.019991,0.000686,0.001372,13,Battery Park City,POINT (-74.01693 40.71186)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835902,way/997819338,Andrew Haswell Green Park,3009.158663,2022,9,9,18,4,False,Autumn,False,2,692556.68290,689547.524236,0.004345,0.008690,140,Lenox Hill East,"POLYGON ((-73.95786 40.75948, -73.95761 40.759..."
835903,way/997819338,Andrew Haswell Green Park,3009.158663,2022,9,9,19,4,False,Autumn,False,20,692556.68290,689547.524236,0.004345,0.086900,140,Lenox Hill East,"POLYGON ((-73.95786 40.75948, -73.95761 40.759..."
835904,way/997819338,Andrew Haswell Green Park,3009.158663,2022,9,9,20,4,False,Autumn,False,9,692556.68290,689547.524236,0.004345,0.039105,140,Lenox Hill East,"POLYGON ((-73.95786 40.75948, -73.95761 40.759..."
835905,way/997819338,Andrew Haswell Green Park,3009.158663,2022,9,9,8,4,False,Autumn,False,19,692556.68290,689547.524236,0.004345,0.082555,140,Lenox Hill East,"POLYGON ((-73.95786 40.75948, -73.95761 40.759..."


In [ ]:
parks_with_zones_dissolved.shape

(835907, 21)

In [ ]:
#parks_with_zones_dissolved.to_file('parks_with_zones_full_details.geojson',driver='GeoJSON')

In [93]:
parks_with_zones_id=parks_with_zones_dissolved[parks_with_zones_dissolved['park_name']=='East River Waterfront Esplanade']

In [97]:
parks_with_zones_id.head(30)

,park_id,park_name,park_area,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness,taxi_zone_ids,taxi_zones,geometry
91725,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,1,10,5,True,Winter,True,5,2.608732e+06,2.437874e+06,0.431823,0.462037,"88, 209, 45, 232","Financial District South, Seaport, Chinatown, ...","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91726,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,1,11,5,True,Winter,True,18,2.997079e+06,2.783506e+06,0.541814,1.727205,"88, 87, 209, 45, 232","Financial District South, Financial District N...","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91727,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,1,12,5,True,Winter,True,8,2.105461e+06,2.020031e+06,0.091964,0.462465,"45, 232","Chinatown, Two Bridges/Seward Park","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91728,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,1,16,5,True,Winter,True,17,2.997079e+06,2.783506e+06,0.541814,1.160972,"88, 87, 209, 45, 232","Financial District South, Financial District N...","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91729,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,1,17,5,True,Winter,True,11,2.737209e+06,2.566351e+06,0.377445,0.765755,"88, 87, 45, 232","Financial District South, Financial District N...","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91730,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,1,19,5,True,Winter,True,32,2.737209e+06,2.566351e+06,0.377445,3.164280,"88, 87, 45, 232","Financial District South, Financial District N...","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91731,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,1,20,5,True,Winter,True,4,2.365330e+06,2.237186e+06,0.256334,0.420703,"209, 45, 232","Seaport, Chinatown, Two Bridges/Seward Park","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91732,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,10,12,0,False,Winter,False,15,2.305346e+06,2.134487e+06,0.480064,1.505012,"88, 87, 209, 232","Financial District South, Financial District N...","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91733,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,10,13,0,False,Winter,False,18,2.737209e+06,2.566351e+06,0.377445,1.185602,"88, 87, 45, 232","Financial District South, Financial District N...","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."
91734,relation/16066428,East River Waterfront Esplanade,42714.567461,2022,1,10,14,0,False,Winter,False,16,2.997079e+06,2.783506e+06,0.541814,1.572900,"88, 87, 209, 45, 232","Financial District South, Financial District N...","MULTIPOLYGON (((-74.00491 40.70460, -74.00499 ..."


In [95]:
parks_with_zones_dissolved['park_id'].nunique()

458

In [96]:
# Check the number of unique park IDs and geometries
print(f"Unique park IDs: {parks_with_zones_dissolved['park_id'].nunique()}")
print(f"Unique geometries: {parks_with_zones_dissolved['geometry'].nunique()}")

Unique park IDs: 458
Unique geometries: 458


In [ ]:
parks_with_zones_id.columns

Index(['park_id', 'park_name', 'park_area', 'year', 'month', 'day', 'hour',
       'day_of_week', 'weekend', 'season', 'isHoliday?', 'geometry',
       'taxi_zone_busyness', 'taxi_zone_id', 'taxi_zone', 'taxi_zone_area',
       'remaining_taxi_zone_area', 'park_proportion', 'park_busyness',
       'taxi_zone_ids', 'taxi_zones'],
      dtype='object')

In [ ]:
parks_with_zones_id_check=parks_with_zones_id.drop(['day_of_week', 'weekend', 'season', 'isHoliday?',
       'taxi_zone_busyness', 'taxi_zone_id', 'taxi_zone', 'taxi_zone_area',
       'remaining_taxi_zone_area', 'park_proportion', 'park_busyness',
       'taxi_zone_ids', 'taxi_zones'],axis=1)

Check individual duplicates

In [ ]:
# Check for duplicates
duplicates = parks_with_zones_id.duplicated(subset=['park_id', 'year', 'month', 'day', 'hour'], keep=False)
print(parks_with_zones_id[duplicates])


Empty GeoDataFrame
Columns: [park_id, park_name, park_area, year, month, day, hour, day_of_week, weekend, season, isHoliday?, geometry, taxi_zone_busyness, taxi_zone_id, taxi_zone, taxi_zone_area, remaining_taxi_zone_area, park_proportion, park_busyness, taxi_zone_ids, taxi_zones, new_geometry]
Index: []


In [98]:
# Inspect geometries before dissolve
print(parks_with_zones_filtered[parks_with_zones_filtered['park_name'] == 'East River Waterfront Esplanade']['geometry'].unique())



<GeometryArray>
[<MULTIPOLYGON (((-74.003 40.706, -74.004 40.706, -74.004 40.706, -74.006 40....>]
Length: 1, dtype: geometry


In [99]:
# Inspect geometries after dissolve
print(parks_with_zones_dissolved[parks_with_zones_dissolved['park_name'] == 'East River Waterfront Esplanade']['geometry'].unique())


<GeometryArray>
[<MULTIPOLYGON (((-74.005 40.705, -74.005 40.705, -74.005 40.704, -74.005 40....>]
Length: 1, dtype: geometry


In [100]:
testing1= parks_with_zones_dissolved[parks_with_zones_dissolved['park_name'] == 'Battery Park']
testing1

,park_id,park_name,park_area,year,month,day,hour,day_of_week,weekend,season,isHoliday?,taxi_zone_busyness,taxi_zone_area,remaining_taxi_zone_area,park_proportion,park_busyness,taxi_zone_ids,taxi_zones,geometry
686149,way/5029111,Battery Park,88454.547122,2022,1,1,10,5,True,Winter,True,19,801633.010772,536269.369405,1.431327,5.208599,"12, 88, 13","Battery Park, Financial District South, Batter...","POLYGON ((-74.01486 40.70120, -74.01493 40.701..."
686150,way/5029111,Battery Park,88454.547122,2022,1,1,11,5,True,Winter,True,13,801633.010772,536269.369405,1.431327,3.876231,"12, 88, 13","Battery Park, Financial District South, Batter...","POLYGON ((-74.01486 40.70120, -74.01493 40.701..."
686151,way/5029111,Battery Park,88454.547122,2022,1,1,12,5,True,Winter,True,6,558230.811320,381321.717076,1.067918,1.842019,"12, 13","Battery Park, Battery Park City","POLYGON ((-74.01486 40.70120, -74.01493 40.701..."
686152,way/5029111,Battery Park,88454.547122,2022,1,1,16,5,True,Winter,True,21,700471.874292,523562.780048,0.556934,4.403799,"88, 13","Financial District South, Battery Park City","POLYGON ((-74.01486 40.70120, -74.01493 40.701..."
686153,way/5029111,Battery Park,88454.547122,2022,1,1,17,5,True,Winter,True,4,700471.874292,523562.780048,0.556934,1.113869,"88, 13","Financial District South, Battery Park City","POLYGON ((-74.01486 40.70120, -74.01493 40.701..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688197,way/5029111,Battery Park,88454.547122,2022,9,9,18,4,False,Autumn,False,3,457069.674840,368615.127718,0.193525,0.580576,13,Battery Park City,"POLYGON ((-74.01486 40.70120, -74.01493 40.701..."
688198,way/5029111,Battery Park,88454.547122,2022,9,9,19,4,False,Autumn,False,24,700471.874292,523562.780048,0.556934,6.173560,"88, 13","Financial District South, Battery Park City","POLYGON ((-74.01486 40.70120, -74.01493 40.701..."
688199,way/5029111,Battery Park,88454.547122,2022,9,9,20,4,False,Autumn,False,7,700471.874292,523562.780048,0.556934,1.524561,"88, 13","Financial District South, Battery Park City","POLYGON ((-74.01486 40.70120, -74.01493 40.701..."
688200,way/5029111,Battery Park,88454.547122,2022,9,9,8,4,False,Autumn,False,14,700471.874292,523562.780048,0.556934,3.388889,"88, 13","Financial District South, Battery Park City","POLYGON ((-74.01486 40.70120, -74.01493 40.701..."


In [101]:
duplicated_rows = parks_with_zones_dissolved[parks_with_zones_dissolved.duplicated()]

#display duplicate rows
print("Duplicate Rows:")
print(duplicated_rows)

Duplicate Rows:
Empty DataFrame
Columns: [park_id, park_name, park_area, year, month, day, hour, day_of_week, weekend, season, isHoliday?, taxi_zone_busyness, taxi_zone_area, remaining_taxi_zone_area, park_proportion, park_busyness, taxi_zone_ids, taxi_zones, geometry]
Index: []


#### Debugging

Exploring the data when aggregated

In [ ]:
park_busyness_details= parks_with_zones_dissolved.groupby(['park_id','park_name','taxi_zone_ids','geometry'])['park_busyness'].sum().reset_index()

In [ ]:
park_busyness_details.shape

(458, 5)

In [ ]:
park_busyness_details.to_file('park_busyness_details_total.geojson',driver='GeoJSON')

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [ ]:
park_busyness_details['park_id'].value_counts()

park_id
node/11457630561    458
Name: count, dtype: int64

In [ ]:
# Set the geometry column explicitly
park_busyness_details = gpd.GeoDataFrame(park_busyness_details, geometry='geometry')

Exploration on Map

In [ ]:
  #column="BoroName",  # make choropleth based on "BoroName" column
    #tooltip="BoroName",  # show "BoroName" value in tooltip (on hover)
    #popup=True,  # show all values in popup (on click)
    #tiles="CartoDB positron",  # use "CartoDB positron" tiles
    #cmap="Set1",  # use "Set1" matplotlib colormap
    #style_kwds=dict(color="black"),  # use black outline

In [ ]:
n= parks_with_zones_dissolved.nunique()
n

park_id                       458
park_name                     435
park_area                     413
year                            1
month                          12
day                            31
hour                           24
day_of_week                     7
weekend                         2
season                          4
isHoliday?                      2
geometry                      484
taxi_zone_busyness            205
taxi_zone_id                  220
taxi_zone                     220
taxi_zone_area                147
remaining_taxi_zone_area      571
park_proportion               570
park_busyness               39253
taxi_zone_ids                 220
taxi_zones                    220
dtype: int64

In [ ]:
park_busyness_details.explore(column="park_busyness", tooltip="park_name",popup=True, legend=True)

Debug

Step 1: Inspect Data for Precision Issues

In [ ]:
# Inspect geometries before dissolve
before_dissolve = parks_with_zones_filtered[parks_with_zones_filtered['park_name'] == 'East River Waterfront Esplanade']['geometry'].unique()
for geom in before_dissolve:
    print(geom)



MULTIPOLYGON (((-74.0033258 40.7056364, -74.0035494 40.705858, -74.0037578 40.7057771, -74.0061365 40.7042534, -74.0070948 40.7036438, -74.0069956 40.7035459, -74.0075179 40.7032199, -74.0076326 40.7031459, -74.0076882 40.7031022, -74.0077299 40.703056, -74.0077553 40.7030218, -74.0077544 40.7030038, -74.0077775 40.7029552, -74.0078126 40.7029135, -74.0078623 40.7028809, -74.0082356 40.702653, -74.0088407 40.7022887, -74.0091882 40.7020809, -74.0092253 40.7020619, -74.0092716 40.7020418, -74.0094947 40.7019645, -74.0097641 40.70188069999998, -74.0100547 40.7018044, -74.0102586 40.7017422, -74.0110892 40.7015437, -74.0112204 40.7015138, -74.0111982 40.7014586, -74.0111862 40.7014615, -74.0111492 40.7014638, -74.0110668 40.7014676, -74.0109606 40.701492, -74.0108482 40.70151709999998, -74.0104073 40.7016246, -74.0100371 40.7017218, -74.0095772 40.701844399999985, -74.0095346 40.7018582, -74.0094962 40.7018708, -74.0093753 40.7019102, -74.0093552 40.7019168, -74.0093258 40.7019263, -74.00

In [ ]:
# Inspect geometries after dissolve
after_dissolve = parks_with_zones_dissolved[parks_with_zones_dissolved['park_name'] == 'East River Waterfront Esplanade']['geometry'].unique()
for geom in after_dissolve:
    print(geom)

MULTIPOLYGON (((-74.0038425 40.705304, -74.0032458 40.704765, -74.0026364 40.70422279999999, -74.0028652 40.704074, -74.0035566 40.7047145, -74.0037713 40.7049092, -74.0040504 40.7051665, -74.00414530000002 40.7051086, -74.0042549 40.7050365, -74.0047929 40.7046706, -74.0052327 40.7043951, -74.0058811 40.7039779, -74.0060214 40.7038839, -74.0060713 40.703925, -74.0062657 40.7037818, -74.006253 40.7037716, -74.0063425 40.7037144, -74.006352 40.7037227, -74.0064957 40.7036162, -74.0065049 40.7036228, -74.0065114 40.7036304, -74.0063669 40.703736, -74.0064394 40.7037992, -74.00659140000002 40.7037016, -74.0075338 40.703124899999985, -74.0076999 40.7028637, -74.00912510000002 40.7019948, -74.0093009 40.7019343, -74.0095772 40.701844399999985, -74.0104073 40.7016246, -74.0108482 40.70151709999998, -74.0110668 40.7014676, -74.0111862 40.7014615, -74.0111982 40.7014586, -74.0112204 40.7015138, -74.0110892 40.7015437, -74.0102586 40.7017422, -74.0100547 40.7018044, -74.0097641 40.7018806999999

Step 2: Simplify Geometries with Higher Tolerance

In [ ]:
parks_with_zones_filtered['geometry'] = parks_with_zones_filtered['geometry'].apply(lambda geom: geom.simplify(0.001))


c:\Users\Justh\anaconda3\envs\comp47360\Lib\site-packages\geopandas\geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Step 3: Ensure Correct Aggregation with Dissolve

In [ ]:
parks_with_zones_dissolved_1 = parks_with_zones_filtered.dissolve(
    by=['park_id', 'park_name', 'park_area', 'year', 'month', 'day', 'hour', 'day_of_week', 'weekend', 'season', 'isHoliday?'],
    aggfunc='sum'
).reset_index()


Step 4: Manually Union Geometries

In [ ]:
from shapely.ops import unary_union

# Manually union geometries to ensure they are unified
def union_geometries(group):
    return unary_union(group.geometry)

parks_with_zones_dissolved_1['geometry'] = parks_with_zones_dissolved_1.groupby(['park_id', 'park_name'])['geometry'].apply(union_geometries).reset_index(drop=True)


In [ ]:
# Verify the result
specific_park = parks_with_zones_dissolved[parks_with_zones_dissolved['park_name'] == 'Battery Park']['geometry'].unique()
for geom in specific_park:
    print(geom)

POLYGON ((-74.01474540000001 40.7015965, -74.0147994 40.7015139, -74.01478360000002 40.70148669999998, -74.0147951 40.70147459999998, -74.0148029 40.7014665, -74.0149056 40.7013344, -74.0148823 40.7013239, -74.0149013 40.70129940000001, -74.0148644 40.7012862, -74.0148349 40.701275399999986, -74.0148011 40.7012617, -74.0148552 40.7011974, -74.0149312 40.7011113, -74.0150255 40.700997, -74.015138 40.7008491, -74.0156798 40.7012755, -74.0157066 40.7012989, -74.0161079 40.7016281, -74.0161415 40.7016586, -74.0165153 40.7020281, -74.0165278 40.7020413, -74.0165358 40.7020497, -74.0168948 40.7024281, -74.0169147 40.7024491, -74.0172279 40.70281509999999, -74.0172629 40.702856, -74.017408 40.7030346, -74.0174956 40.7031607, -74.017523 40.7032074, -74.0175512 40.7032631, -74.0175775 40.7033174, -74.01759860000001 40.703368799999986, -74.0176203 40.7034289, -74.0176328 40.703461, -74.0176453 40.70349779999999, -74.017663 40.7035982, -74.0176668 40.7036167, -74.017674 40.7037137, -74.0176739 40

In [ ]:
park_with_zones_before_dissolve=parks_with_zones.nunique()
park_with_zones_before_dissolve

park_id                        458
park_name                      435
website                        153
park_area                      413
geometry                       458
index_right                 113735
taxi_zone_id                    65
year                             1
month                           12
day                             31
hour                            24
day_of_week                      7
weekend                          2
season                           4
isHoliday?                       2
taxi_zone_busyness             166
taxi_zone                       65
taxi_zone_area                  65
remaining_taxi_zone_area       477
park_proportion                477
park_busyness                21497
dtype: int64

In [ ]:
parks_with_zones_after_dissolve= parks_with_zones_dissolved.nunique()
parks_with_zones_after_dissolve

park_id                       458
park_name                     435
park_area                     413
year                            1
month                          12
day                            31
hour                           24
day_of_week                     7
weekend                         2
season                          4
isHoliday?                      2
geometry                      484
taxi_zone_busyness            205
taxi_zone_id                  220
taxi_zone                     220
taxi_zone_area                147
remaining_taxi_zone_area      571
park_proportion               570
park_busyness               39253
taxi_zone_ids                 220
taxi_zones                    220
dtype: int64